# Analizando la base de datos "Quién es quién en los precios".

Partimos de la base de datos "Quién es quién en los Precios" de la Profeco. Trataremos de responder algunas preguntas sencillas. El código utilizado se encuentra al final del notebook.

## Procesamiento de datos

¿Cuántos registros hay?

**Respuesta:**
62530715

¿Cuántas categorías?

**Respestas:** 42

¿Cuántas cadenas comerciales están siendo monitoreadas?

**Respuesta:** 706

¿Cómo podrías determinar la calidad de los datos? ¿Detectaste algún tipo de
inconsistencia o error en la fuente?

**Respuesta:** 
- Existen valores nulos en diferentes columnas. 
- Algunos valores en la columna "estado" no pertenecen a estados. 
- Existe las categoría "none" y "categoría" en la columan "categoría".

¿Cuáles son los productos más monitoreados en cada entidad?

**Respuesta:**

<img src="files/1.jpeg">

¿Cuál es la cadena comercial con mayor variedad de productos monitoreados?

**Respuesta:**

<img src="files/2.jpeg">

### Análisis exploratorio

¿Cuál es la ciudad más cara del país?¿Cuál es la más barata?

**Respuesta:**

*Más cara:* Mineral de la reforma

*Más barata:* Disrito Federal

¿Cuál es el estado más caro y en qué mes?

**Respuesta:**

Querétaro en el mes de Octubre.

¿Cuáles son los principales riesgos de hacer análisis de series de tiempo con estos datos?

**Respuesta:**
Los periodos de tiempo pueden no ser útiles para hacer predicciones. Los datos podrían presentar patrones pero puede no existir correlación entre ellos.

In [44]:
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
import pyspark.sql.functions as F

In [2]:
import os 
import sys

def configure_spark(spark_home=None, pyspark_python=None):
    spark_home = spark_home or "/path/to/default/spark/home"
    os.environ['SPARK_HOME'] = spark_home
    os.environ['JAVA_HOME'] = 'C:\Program Files\Java\jre1.8.0_261'

    # Add the PySpark directories to the Python path:
    sys.path.insert(1, os.path.join(spark_home, 'python'))
    sys.path.insert(1, os.path.join(spark_home, 'python', 'pyspark'))
    sys.path.insert(1, os.path.join(spark_home, 'python', 'build'))

    # If PySpark isn't specified, use currently running Python binary:
    pyspark_python = pyspark_python or sys.executable
    os.environ['PYSPARK_PYTHON'] = pyspark_python

configure_spark('C:\spark\spark-3.0.1-bin-hadoop2.7')

In [3]:
spark = SparkSession \
     .builder \
     .appName("OPItest") \
     .getOrCreate() 

In [4]:
profeco = spark.read.csv('all_data.csv', header=True)

In [22]:
profeco.printSchema()

root
 |-- producto: string (nullable = true)
 |-- presentacion: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- catalogo: string (nullable = true)
 |-- precio: string (nullable = true)
 |-- fechaRegistro: string (nullable = true)
 |-- cadenaComercial: string (nullable = true)
 |-- giro: string (nullable = true)
 |-- nombreComercial: string (nullable = true)
 |-- direccion: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- latitud: string (nullable = true)
 |-- longitud: string (nullable = true)



In [41]:
profeco.count()

62530715

In [42]:
profeco.select("categoria").distinct().count()

42

In [43]:
profeco.select("cadenaComercial").distinct().count()

706

In [46]:
profeco.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in profeco.columns]).show()

+--------+------------+-----+---------+--------+------+-------------+---------------+----+---------------+---------+------+---------+-------+--------+
|producto|presentacion|marca|categoria|catalogo|precio|fechaRegistro|cadenaComercial|giro|nombreComercial|direccion|estado|municipio|latitud|longitud|
+--------+------------+-----+---------+--------+------+-------------+---------------+----+---------------+---------+------+---------+-------+--------+
|       0|           0|    0|   887338|     228|     0|            0|           1184| 287|              0|        0| 15054|    15054|  36814|   36814|
+--------+------------+-----+---------+--------+------+-------------+---------------+----+---------------+---------+------+---------+-------+--------+



In [ ]:
profeco.groupBy("estado" "fechaRegistro").agg(F.avg("precios").show()

In [5]:
from pyspark.sql.window import Window

In [18]:
from pyspark.sql import SQLContext
sqlCtx = SQLContext(spark)

In [6]:
w1 = Window.partitionBy("producto").orderBy("estado")

In [40]:
grouped = profeco.groupBy("estado", "producto").count()
w2 = Window.partitionBy("estado").orderBy(F.desc("count"))
grouped.withColumn("order", F.row_number().over(w2)).where(col("order") == 1).show()

+--------------------+--------------------+------+-----+
|              estado|            producto| count|order|
+--------------------+--------------------+------+-----+
|        QUINTANA ROO|                 FUD| 34846|    1|
|          NUEVO LEÓN|   DETERGENTE P/ROPA| 50307|    1|
|             SINALOA|            REFRESCO| 33115|    1|
|             TABASCO|            REFRESCO| 28754|    1|
|     BAJA CALIFORNIA|            REFRESCO| 37243|    1|
|            TLAXCALA|            REFRESCO| 43904|    1|
|COAHUILA DE ZARAGOZA|                 FUD| 28613|    1|
|                null|LECHE ULTRAPASTEU...|   804|    1|
|       ESQ. SUR 125"|  PAN BLANCO BOLILLO|   130|    1|
| COL. EDUARDO GUERRA|            REFRESCO|   275|    1|
|             CHIAPAS|            REFRESCO| 14452|    1|
|VERACRUZ DE IGNAC...|            REFRESCO| 14576|    1|
|              SONORA|            REFRESCO| 34731|    1|
|             YUCATÁN|LECHE ULTRAPASTEU...| 35991|    1|
| MICHOACÁN DE OCAMPO|   DETERG

In [50]:
grouped1 = profeco.groupBy("cadenaComercial", "producto").count()
w3 = Window.partitionBy("cadenaComercial").orderBy(F.desc("count"))
grouped1.withColumn("order", F.row_number().over(w3)).where(F.col("order") == 1).show()

+--------------------+--------------------+-----+-----+
|     cadenaComercial|            producto|count|order|
+--------------------+--------------------+-----+-----+
|JULIO CEPEDA JUGU...|              VARIOS| 8945|    1|
|LIBRERIA DE SANCH...|LIBRO DE TEXTO DE...|  111|    1|
| MARISCOS LA SEPTIMA|                ATUN|    8|    1|
|PAPELERIA EL TINTERO|         COLOR LARGO| 1706|    1|
|PAPELERIA Y LIBRE...|LIBRO DE TEXTO DE...|  135|    1|
|     PLAZA DEL LIBRO|LIBRO DE TEXTO DE...|   24|    1|
|           WOOLWORTH|             MUÑECAS| 3227|    1|
|MERCADO ALONSO FE...|           CARNE RES|   81|    1|
|           MEGA TOYS|              VARIOS|  553|    1|
|     MERCADO LA CRUZ|   DETERGENTE P/ROPA| 1128|    1|
|              SIGUBA|             MUÑECAS|  382|    1|
|     ZAPATERIA PAKAR|               TENIS|  594|    1|
|    BODEGAS VICTORIA|           CARNE RES|   21|    1|
| MARISCOS VILLA AZUL|             CAMARON|   68|    1|
|     MINISUPER INALA|      CHILES EN LATA|   16

In [61]:
profeco.registerTempTable("profeco")
sqlCtx.sql("select estado, precio, avg(precio) over (partition by precio) as avg from profeco group by estado, precio").show()

+--------------------+------+------------------+
|              estado|precio|               avg|
+--------------------+------+------------------+
|                null|  10.7|10.699999999999994|
|             YUCATÁN|  10.7|10.699999999999994|
|           CHIHUAHUA|  10.7|10.699999999999994|
|              OAXACA|  10.7|10.699999999999994|
|             TABASCO|  10.7|10.699999999999994|
|            CAMPECHE|  10.7|10.699999999999994|
|     SAN LUIS POTOSÍ|  10.7|10.699999999999994|
|     BAJA CALIFORNIA|  10.7|10.699999999999994|
|          NUEVO LEÓN|  10.7|10.699999999999994|
|           ZACATECAS|  10.7|10.699999999999994|
| BAJA CALIFORNIA SUR|  10.7|10.699999999999994|
|             MORELOS|  10.7|10.699999999999994|
|              SONORA|  10.7|10.699999999999994|
|COAHUILA DE ZARAGOZA|  10.7|10.699999999999994|
|              MÉXICO|  10.7|10.699999999999994|
| MICHOACÁN DE OCAMPO|  10.7|10.699999999999994|
|             NAYARIT|  10.7|10.699999999999994|
|            TLAXCAL

In [63]:
profeco.groupBy("municipio").agg({"precio": "avg"}).orderBy().show()

+--------------------+------------------+
|           municipio|       avg(precio)|
+--------------------+------------------+
|OJINAGA          ...| 198.1150955742386|
|SANTA LUCIA DEL C...|11.621656050955414|
|TLAHUAC          ...| 13.38072744449693|
|          XOCHIMILCO| 230.7388879712314|
|GUADALUPE        ...| 268.9884006379839|
|GUADALAJARA      ...| 340.6231118330885|
|CUAUTITLAN IZCALL...| 692.2871769366495|
|  CUAUTITLÁN IZCALLI| 867.1438786942692|
|              MÉRIDA| 610.5113959312602|
|MIGUEL HIDALGO   ...| 293.2165905802715|
|MERIDA           ...| 593.8746627387962|
|CAMPECHE         ...|387.62871000175943|
|           TULTITLÁN|212.48612204769518|
|TLALPAN          ...|386.17056708765085|
|                LEÓN| 655.3993888795545|
|               TEPIC|  817.983152734674|
|SAN PEDRO GARZA G...| 1567.897594574927|
|        AZCAPOTZALCO| 454.1550908268682|
|          CP. 27280"|204.69219159008608|
|            CÁRDENAS| 727.2738708006151|
+--------------------+------------

In [68]:
profeco.groupBy("municipio").agg({"precio": "avg"}).sort(F.desc("avg(precio)")).show()

+--------------------+------------------+
|           municipio|       avg(precio)|
+--------------------+------------------+
|MINERAL DE LA REF...| 2351.352747252748|
|MINERAL DE LA REF...|       1794.859375|
|SAN PEDRO GARZA G...| 1567.897594574927|
|METEPEC          ...| 1434.393055830713|
|         TLAQUEPAQUE|1081.5867608036938|
|SAN PEDRO GARZA G...| 946.3962263344739|
|              CARMEN| 923.2896217907679|
|DURANGO          ...| 917.7662001202675|
|             DURANGO| 915.5655683192243|
|            VERACRUZ| 904.9979154132332|
|              COLIMA| 897.0281669134549|
|             METEPEC| 890.9500353179583|
|             TORREÓN|  889.800787581794|
|     OTHÓN P. BLANCO| 889.7990291493196|
|COLIMA           ...| 867.7465286727949|
|  CUAUTITLÁN IZCALLI| 867.1438786942692|
|     PACHUCA DE SOTO| 857.5933525629733|
|     SAN LUIS POTOSÍ| 831.4011677720871|
|PACHUCA DE SOTO  ...|  824.302926808676|
|            CAMPECHE| 818.7400068117165|
+--------------------+------------

In [69]:
profeco.groupBy("municipio").agg({"precio": "avg"}).sort(F.asc("avg(precio)")).show()

+--------------------+------------------+
|           municipio|       avg(precio)|
+--------------------+------------------+
|           municipio|              null|
|    DISTRITO FEDERAL|1.6386363636363637|
|SANTA CATARINA   ...|  9.60223404255319|
|      SANTA CATARINA|10.033333333333335|
|SAN MATEO ATENCO ...|              10.7|
|TLAJOMULCO DE ZUÑ...|10.742156862745098|
|GENERAL ESCOBEDO ...|10.976972072513467|
|ATIZAPAN DE ZARAG...|10.983198653198652|
|SANTA LUCIA DEL C...|11.621656050955414|
|SANTA LUCIA DEL C...|12.021818181818182|
|    GENERAL ESCOBEDO|12.050467289719625|
|CUAJIMALPA DE MOR...| 13.34045584045584|
|TLAHUAC          ...| 13.38072744449693|
|MILPA ALTA       ...|13.448717948717949|
|          MILPA ALTA|              15.0|
|CUAJIMALPA DE MOR...|              15.0|
|             TLÁHUAC|              15.0|
|ATIZAPÁN DE ZARAGOZA|16.266819172113287|
|           HUAMANTLA| 29.61502898916056|
|             URUAPAN| 40.13605195224261|
+--------------------+------------

In [71]:
d = profeco.groupBy("estado", "fechaRegistro").agg({"precio":"avg"}).sort(F.desc("avg(precio)")).show()

+----------------+--------------------+-----------+
|          estado|       fechaRegistro|avg(precio)|
+----------------+--------------------+-----------+
|       QUERÉTARO|2015-10-06 17:01:...|   151199.0|
|       QUERÉTARO|2015-10-13 15:10:...|   151199.0|
|      NUEVO LEÓN|2015-10-05 15:31:...|   131999.2|
|DISTRITO FEDERAL|2015-10-02 14:13:...|   131999.2|
|      NUEVO LEÓN|2015-09-28 16:21:...|   131999.2|
|      NUEVO LEÓN|2015-10-19 14:04:...|   129999.0|
|      NUEVO LEÓN|2015-10-12 13:59:...|   129999.0|
|      NUEVO LEÓN|2015-10-26 13:52:...|   129999.0|
|      NUEVO LEÓN|2015-10-05 13:45:...|   129999.0|
|      GUANAJUATO|2015-10-12 12:02:...|  123749.25|
|          PUEBLA|2015-10-02 14:49:...|  121439.26|
|      NUEVO LEÓN|2015-10-12 16:08:...|  121439.26|
|      NUEVO LEÓN|2015-10-19 16:19:...|  121439.26|
|          PUEBLA|2015-10-09 14:07:...|  121439.26|
|      NUEVO LEÓN|2015-10-26 16:26:...|  121439.26|
|          PUEBLA|2015-10-16 15:04:...|  121439.26|
|          P